# Research on reliability of debitors


The customer is the credit department of the bank. It is necessary to figure out whether the marital status, number of children, education and goals of the client's loan affect the fact that the loan is repaid on time. Input data from the bank — statistics on the solvency of customers.
The results of the research will be taken while we building a credit scoring model — a special system that evaluates the ability of a potential borrower to repay a loan to a bank.

Let's import all the necessary libraries, open data source file, and take a look on our data:


In [2]:
import pandas as pd
from IPython.display import display
from pandas.api.types import CategoricalDtype
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="Рассчитано")
pd.options.display.float_format = "{:,.2f}".format

In [3]:
data = pd.read_csv('./datasets/data.csv')
print(data.info())
data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,"-8,437.67",42,высшее,0,женат / замужем,0,F,сотрудник,0,"253,875.64",покупка жилья
1,1,"-4,024.80",36,среднее,1,женат / замужем,0,F,сотрудник,0,"112,080.01",приобретение автомобиля
2,0,"-5,623.42",33,Среднее,1,женат / замужем,0,M,сотрудник,0,"145,885.95",покупка жилья
3,3,"-4,124.75",32,среднее,1,женат / замужем,0,M,сотрудник,0,"267,628.55",дополнительное образование
4,0,"340,266.07",53,среднее,1,гражданский брак,1,F,пенсионер,0,"158,616.08",сыграть свадьбу
5,0,-926.19,27,высшее,0,гражданский брак,1,M,компаньон,0,"255,763.57",покупка жилья
6,0,"-2,879.20",43,высшее,0,женат / замужем,0,F,компаньон,0,"240,525.97",операции с жильем
7,0,-152.78,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,"135,823.93",образование
8,2,"-6,929.87",35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,"95,856.83",на проведение свадьбы
9,0,"-2,188.76",41,среднее,1,женат / замужем,0,M,сотрудник,0,"144,425.94",покупка жилья для семьи


We can see strange data in a column named `days_employed` - values are negative, and we need to change it to positive. The `education` column contains rows in different registers with the types of education, so let's convert them to the same register. After that, we can change the data in the `family_status` column, because we need to know whether our debtor is married, in an informal relationship, or widowed/divorced. And we will also make some changes to the data in the `purpore` column: we will conduct a lexical research about the purpose of the loan, find and assign typical categories for each purpose. After basic research, we will try to expand our analysis by adding a check of the relationship between the probability of debt repayment and years spent at work. To do this, it will be nice to recalculate the days from the column into years.<br>

In our research such data as `education_id`, `family_status_id`, `gender` will not be needed. Just drop it.

Some columns we will only need to substitute data in other columns. There are `dob_year` and `income_type`.

Let's check, how many NaNs do we have in data?

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

There are two columns in dataframe: `days_employed` and `total_income` with NaNs. We can subusitiute omissions in `total_income` by median value of grouped data by type in `income_type` field. Before that, we must make sure, that all values in column `income_type` are valid.

In [5]:
data.income_type.value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Well, since we don't have invalid or duplicate values, we can group the data by them and calculate the median value of the group. After that, we will fill in missing values.

In [6]:
data.total_income = data.total_income.fillna(data.groupby(['income_type'])['total_income'].transform('median'))
print(f'NaN values in column "total_income": {data.total_income.isna().sum()}')
data.total_income.describe()

NaN values in column "total_income": 0


count      21,525.00
mean      165,225.32
std        98,043.67
min        20,667.26
25%       107,798.17
50%       142,594.40
75%       195,549.94
max     2,265,604.03
Name: total_income, dtype: float64

When we took a look at the "education" column, we saw that this field had the same values written in different ways.

Let's transform that field. First, we need to observe all values in column:

In [7]:
data.education.value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Some words represented in that column are written entirely in uppercase, starting with a capital letter or consisting entirely of small letters. let's bring them all to the lower case.

In [8]:
data.education = data.education.str.lower()
data.education.value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Now it looks much better. But, we definitely have categorial type of data in that column. Let's transform and arrange them into the correct order: 'начальное' -> 'среднее' -> 'неоконченное высшее' -> 'высшее' -> 'ученая степень'.

In [9]:
data.education = data.education.astype('category')
data.education.head()

0     высшее
1    среднее
2    среднее
3    среднее
4    среднее
Name: education, dtype: category
Categories (5, object): ['высшее', 'начальное', 'неоконченное высшее', 'среднее', 'ученая степень']

In [10]:
#order in this list will define order of category
ed_order = ['начальное', 'среднее', 'неоконченное высшее', 'высшее', 'ученая степень']
covered_type = CategoricalDtype(categories=ed_order,
                                ordered=True)
#change this category order, as we can see in the last string (Categories (5, object): etc.) of output.
data.education = data.education.cat.reorder_categories(ed_order, ordered=True)
data.education.head()

0     высшее
1    среднее
2    среднее
3    среднее
4    среднее
Name: education, dtype: category
Categories (5, object): ['начальное' < 'среднее' < 'неоконченное высшее' < 'высшее' < 'ученая степень']

Task completed. Data type now "Categorial"

Now time to make change values in column `family_status`. That field contains some data, that means 'Single' or 'In an unformal relatioship', 'Married' and 'Widow(er)/divorced'. Convert the values to the ones specified earlier. Take look at current meanings.

In [11]:
data.family_status.value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

As we could see earlier `family_status` there is no single NaN, no hidden duplicates. Change `Не женат / не замужем` and `гражданский брак` to `single`, `женат / замужем` to `married` and `в разводе` and `вдовец / вдова` to `widow(er)/divorced`

In [12]:
def change_fam_stat(family_status):
    if family_status in ['гражданский брак', 'Не женат / не замужем']:
        return 'single'
    elif family_status in ['женат / замужем']:
        return 'married'
    else:
        return 'widow(er)/divorced'

In [13]:
data.family_status = data.family_status.progress_apply(change_fam_stat)
data.family_status.value_counts()

Рассчитано: 100%|██████████| 21525/21525 [00:00<00:00, 906923.23it/s]


married               12380
single                 6990
widow(er)/divorced     2155
Name: family_status, dtype: int64

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   children          21525 non-null  int64   
 1   days_employed     19351 non-null  float64 
 2   dob_years         21525 non-null  int64   
 3   education         21525 non-null  category
 4   education_id      21525 non-null  int64   
 5   family_status     21525 non-null  object  
 6   family_status_id  21525 non-null  int64   
 7   gender            21525 non-null  object  
 8   income_type       21525 non-null  object  
 9   debt              21525 non-null  int64   
 10  total_income      21525 non-null  float64 
 11  purpose           21525 non-null  object  
dtypes: category(1), float64(2), int64(5), object(4)
memory usage: 1.8+ MB


Ok, all done, looks good

### Processing of abnormal values

In field `days_employed` we have negative values. We can change it with method `abs`, but we don't need this field in our research, and just drop it.

In [15]:
data.drop(columns=['days_employed'], axis=1, inplace=True)

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   children          21525 non-null  int64   
 1   dob_years         21525 non-null  int64   
 2   education         21525 non-null  category
 3   education_id      21525 non-null  int64   
 4   family_status     21525 non-null  object  
 5   family_status_id  21525 non-null  int64   
 6   gender            21525 non-null  object  
 7   income_type       21525 non-null  object  
 8   debt              21525 non-null  int64   
 9   total_income      21525 non-null  float64 
 10  purpose           21525 non-null  object  
dtypes: category(1), float64(1), int64(5), object(4)
memory usage: 1.7+ MB


By the way, we also don't need fields `education_id`, `family_status_id` and `gender`. Delete'em all.

In [17]:
data.drop(columns=['education_id', 'family_status_id', 'gender'], axis=1, inplace=True)

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   children       21525 non-null  int64   
 1   education      21525 non-null  category
 2   family_status  21525 non-null  object  
 3   income_type    21525 non-null  object  
 4   debt           21525 non-null  int64   
 5   total_income   21525 non-null  float64 
 6   purpose        21525 non-null  object  
dtypes: category(1), float64(1), int64(2), object(3)
memory usage: 1.0+ MB


Now time to count children.

In [19]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

20 children? What? I think, that, of course, mistake and we can change that meaning to 2. And mystical `-1` kid. We can't do anything about is, and will delete these rows.

In [20]:
data.loc[data['children'] == 20, 'children'] = 2

In [21]:
data = (data.loc[data['children'] != -1]).copy()

In [23]:
print(data.children.value_counts())
data.head()

0    14149
1     4818
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


,children,education,family_status,income_type,debt,total_income,purpose
0,1,высшее,married,сотрудник,0,"253,875.64",покупка жилья
1,1,среднее,married,сотрудник,0,"112,080.01",приобретение автомобиля
2,0,среднее,married,сотрудник,0,"145,885.95",покупка жилья
3,3,среднее,married,сотрудник,0,"267,628.55",дополнительное образование
4,0,среднее,single,пенсионер,0,"158,616.08",сыграть свадьбу


In [ ]:
data['children'].unique()

<div class="alert alert-success">
<b>✔️ Комментарий ревьюера v1:</b> 
<br>👏🏻👏🏻👏🏻 Отличная работа с аномальными значениями!
</div>

### Изменение типов данных

**Задание 13. Замените вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [ ]:
data['total_income'] = data['total_income'].astype(int)

<div class="alert alert-success">
<b>✔️ Комментарий ревьюера v1:</b> 
<br>Тут всё верно! Хорошо было бы вывести полученный результат для точной проверки.
</div>

### Обработка дубликатов

**Задание 14. Выведите на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалите их.**

In [ ]:
data.duplicated().sum()

In [ ]:
data = data.drop_duplicates()

<div class="alert alert-warning">
<b>⚠️ Комментарий ревьюера v1:</b> 
<br>Супер! От дубликатов тоже избавились, но рекомендую проверять после выполнения действий.
</div>

**Задание 15. Обработайте неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите их к нижнему регистру. Проверьте остальные столбцы.**

In [ ]:
data['education'] = data['education'].str.lower()

### Категоризация данных

**Задание 16. На основании диапазонов, указанных ниже, создайте в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используйте собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [ ]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [ ]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Задание 17. Выведите на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [ ]:
data['purpose'].unique()

**Задание 18. Создайте функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используйте собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучите данные в столбце `purpose` и определите, какие подстроки помогут вам правильно определить категорию.**

In [ ]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [ ]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

<div class="alert alert-warning">
<b>⚠️ Комментарий ревьюера v1:</b> Выведи, пожалуйста, полученные резульатты.

## Исследуйте данные и ответьте на вопросы

Напомним состав данных:
* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

Помимо этого были добавлены столбцы, с помощью которых мы категоризировали клиентов по уровню дохода и уточнили категории целей взятия кредита:
* total_income_category - категории дохода
* purpose_category цели взятия кредита

Предварительные гипотезы исследования:
* наличие детей и их количество влияют на вероятность возврата кредита
* семейное положение влияет на вероятность возврата кредита
* уровень дохода влияет на вероятность возврата кредита
* цель получения кредита влияет на вероятность возврата кредита

Данные для исследования предварительно проверены, сгруппированы и категоризованы.

**Задание 19. Есть ли зависимость между количеством детей и возвратом кредита в срок?**

### Зависимость между количеством детей и возвратом кредита в срок

Для решения задачи импортируем необходимые библиотеки - `seaborn`, `matplotlib`, `scipy.stat`. 

Сформируем новый датафрейм сгруппировав данные по количеству детей в семье. Количество детей будет не индексом а столбцом, чтобы иметь возможность рассчитать коэффициент корреляции. Здесь же рассчитаем среднее значение просроченных кредитов и переведем значения в проценты. 

Рассчитаем коэффициент корелляции Спирмена и p-value, чтобы понять - есть зависимость или нет и ее статистическую достоверность. Коэффициент Спирмена был выбран нами из-за того, что будет проверено небольшое количество данных и шкала "Дети" - интервальная (порядковая) с одной стороны и шкала "среднее количество просроченных кредитов" непрерывная (метрическая) с другой. 

Для лучшего представления данных построим столбчатую диаграмму. 

In [ ]:
# Ваш код будет здесь. Вы можете создавать новые ячейки.
# Импортируем библиотеки для построения диаграмм и вычисления статистики, определяем стиль диаграмм, 
# задаем константу для печати разделителя
import seaborn
import matplotlib.pyplot as plt
# Устанавливаем стиль
plt.style.use('ggplot')

# Изменяем размер диаграммы
plt.rcParams['figure.figsize']=(12,8)
seaborn.set(font_scale=1.3)

from scipy.stats import pearsonr, spearmanr

separator = '____________________________________________________________________________________________'

# Формируем датафрейм, сгруппировав данные по количеству детей в семье, сделав количество детей столбцом.
# Так же рассчитываем отношение невозвращенных кредитов к общему количеству кредитов в группе.
chld_stat = data.groupby('children', as_index=False)['debt'].mean()

# Домножим полученные значения в столбце 'debt' на сто и округлим до двух знаков, получив процент невозврата 
# кредитов по группе с точностью до сотых процента.  

chld_stat['debt'] = round(chld_stat['debt'] * 100, ndigits=2)

print('Процент невозвратов по группам клиентов с n количеством детей: \n')
print(chld_stat)
print(separator)
print()

# Рассчитаем по выборке коэффициент корреляции Спирмена и p-value
r, p = spearmanr(chld_stat['children'], chld_stat['debt'])
print(f'Для выборки коэффициент Спирмена = {r:.02f}, p-value = {p:.03}.')
print(separator)
print()

print('Построим диаграмму:')
# Построим диаграмму для более наглядного представления
ax = seaborn.barplot(data=chld_stat, x='children', y = 'debt')
ax.set(xlabel='Количество детей',
       ylabel='Процент просрочек',
       title='Процент просрочек у семей с детьми \n')

plt.draw()

**Предварительный вывод:** 

Исходя из полученных значений можно предположить отсутствие зависимости между невыплатой задолженности и фактом возрастания количества детей в семье (коэффициент корреляции Спирмена лежит около 0, значение p-value говорит об отсутствии статистической значимости). Однако, на данные мог повлиять выброс в семьях с кол-вом детей = 5. Исследуем данные, чтобы понять сколько семей с количеством детей равным пяти встречается в датасете. Если общее количество невелико, можно отбросить эти данные.

<div class="alert alert-success">
<b>✔️ Комментарий ревьюера v1:</b> 
<br>Отлично!
</div>

In [ ]:
ch_5 = data.loc[data['children'] > 4]['children'].count()
print(f'Семей с детьми более 4: {ch_5}')

Действительно, таких семей только 9 из более чем 23 тысяч. Пересчитаем данные, отбросив данные о семьях с числом детей = 5. 

In [ ]:
#Создадим новый датасет без семей с количеством детей = 5
data_without_5 = data.loc[data['children'] <5 ]

# Далее повторяем вычисления с новым датасетом
# Формируем датафрейм, сгруппировав данные по количеству детей в семье, сделав количество детей столбцом.
# Так же рассчитываем отношение невозвращенных кредитов к общему количеству кредитов в группе.
chld_stat = data_without_5.groupby('children', as_index=False)['debt'].mean()

# Домножим полученные значения в столбце 'debt' на сто и округлим до двух знаков, получив процент невозврата 
# кредитов по группе с точностью до сотых процента.  
chld_stat['debt'] = round(chld_stat['debt'] * 100, ndigits=2)

print('Процент невозвратов по группам клиентов с n количеством детей: \n')
print(chld_stat)
print(separator)
print()

# Рассчитаем по выборке коэффициент корреляции Спирмена и p-value
r, p = spearmanr(chld_stat['children'], chld_stat['debt'])
print(f'Для выборки коэффициент Спирмена = {r:.02f}, p-value = {p:.03}.')
print(separator)
print()

print('Построим диаграмму:')
# Построим диаграмму для более наглядного представления
ax = seaborn.barplot(data=chld_stat, x='children', y = 'debt')
ax.set(xlabel='Количество детей',
       ylabel='Процент просрочек',
       title='Процент просрочек у семей с детьми \n')

plt.draw()

**Вывод:**

Действительно, корреляция изменилась: коэффициент Спирмена перешел в категорию "заметная", что выглядит более убедительно с точки зрения данных и их графического представления. Но p-value остается слишком высоким, чтобы наша гипотеза о  повышении вероятности невыплаты кредита от возрастания количества детей в семье была принята. Вероятность ошибки в таком предположении весьма существена, хотя говорить о ее полном отсутствии нельзя. 

Таким образом, можно сделать вывод о том, что количество детей в семье **мало влияет** на вероятность выплаты кредита, но, при этом, надо отметить, что наименьшим процент просрочки будет в семьях без детей, видимо из-за большей предсказуемости семейного бюджета.

<div class="alert alert-success">
<b>✔️ Комментарий ревьюера v1:</b> 
<br>Верно! Отдельный плюс за визуализацию, молодец!
</div>

**Задание 20. Есть ли зависимость между семейным положением и возвратом кредита в срок?**

### Зависимость между семейным положение и возвратом кредита в срок

Для решения задачи сформируем новый датафрейм, сгруппировав данные по виду семейного положения заемщика. Семейное положение будет не индексом а столбцом, чтобы иметь возможность рассчитать коэффициент корреляции. Здесь же рассчитаем среднее значение просроченных кредитов и переведем значения в проценты.

Рассчитаем коэффициент корелляции Спирмена и p-value, чтобы понять - есть зависимость или нет и ее статистическую достоверность. Коэффициент Спирмена был выбран нами из-за того, что будет проверено небольшое количество данных и шкала "Семейное положение" - категориальная с одной стороны, а шкала "среднее количество просроченных кредитов" непрерывная (метрическая) с другой.

Для лучшего представления данных построим столбчатую диаграмму.

In [ ]:
# Ваш код будет здесь. Вы можете создавать новые ячейки.

# Формируем датафрейм, сгруппировав данные по семейному положению, сделав сделав его столбцом.
# Так же рассчитываем отношение невозвращенных кредитов к общему количеству кредитов в группе.
fam_stat = data.groupby('family_status', as_index=False)['debt'].mean()

# Домножим полученные значения в столбце 'debt' на сто и округлим до двух знаков, получив процент невозврата 
# кредитов по группе с точностью до сотых процента.  

fam_stat['debt'] = round(fam_stat['debt'] * 100, ndigits=2)

print('Процент невозвратов кредитов в зависимости от семейного положения: \n')
print(fam_stat)
print(separator)
print()

# Рассчитаем по выборке коэффициент ранговой корреляции Спирмена и p-value
r, p = spearmanr(fam_stat['family_status'], fam_stat['debt'])
print(f'Для выборки коэффициент Спирмена = {r:.02f}, p-value = {p:.03}.')
print(separator)
print()

print('Построим диаграмму:')
plt.figure(figsize=(10, 10))
plt.xlabel('Процент просрочек')
plt.ylabel('')
plt.title('Процент просрочек в зависимости от семейного положения \n', loc='left')
plt.barh(y=fam_stat['family_status'], width=fam_stat['debt'])
plt.draw()

**Предварительный вывод:** 
Если верить исключительно цифрам, то корреляция мала, p-value не позволяет говорить о статистической достоверности гипотезы о наличии зависимости вероятности невыплаты кредита от семейного положения. Но, если посмотреть на графическое представление данных, то можно заметить, что две группы допускают б**о**льшее количество невыплат, чем три других. Отличаются эти две группы по одному признаку - люди входящие в них не состоят в официально оформленных отношениях. К тому же, есть вероятность, что указавшие себя как не женатые / не замужем, так же находятся в неоформленных отношениях. Указавшие же себя как "состоящие в гражданском браке" с юридической точки зрения входят в группу "не женат / не замужем". Попробуем объединить группы по признаку: состоит ли заемщик в официально зарегистрированных отношениях или нет.

<div class="alert alert-success">
<b>✔️ Комментарий ревьюера v1:</b> 
<br>Тут всё правильно
</div>

In [ ]:
# Выведем идентификаторы групп семейного положения
print(data.groupby('family_status').agg({'family_status_id':['unique', 'count']}))

In [ ]:
# Напишем функцию для дополнительно категоризации семейного положения заемщиков
def categorize_family_status(family_status_id):
    try:
        if family_status_id in [4, 1]:
            return 'Не состоит в оф. отношениях'
        else:
            return 'Состоит/ранее состоял в оф. отношениях'
    except:
        return 'Неопределенная категория'

<div class="alert alert-success">
<b>✔️ Комментарий ревьюера v1:</b> Здесь верно, можно было кейсами прописать также.

In [ ]:
# Применим функцию по столбцу family_status_id
data['family_status_category'] = data['family_status_id'].apply(categorize_family_status)
data.head()

In [ ]:
# Выполним расчеты с помощью новой группировки

# Формируем датафрейм, сгруппировав данные по типу семейного положения, сделав сделав его столбцом.
# Так же рассчитываем отношение невозвращенных кредитов к общему количеству кредитов в группе.
fam_stat = data.groupby('family_status_category', as_index=False)['debt'].mean()

# Домножим полученные значения в столбце 'debt' на сто и округлим до двух знаков, получив процент невозврата 
# кредитов по группе с точностью до сотых процента.  

fam_stat['debt'] = round(fam_stat['debt'] * 100, ndigits=2)

print('Процент невозвратов кредитов в зависимости от семейного положения: \n')
print(fam_stat)
print(separator)
print()

print('Построим диаграмму:')
ax = seaborn.barplot (data=fam_stat, y='family_status_category', x='debt')
ax.set(xlabel='Процент просрочек',
       ylabel='',
       title='Процент просрочек в зависимости от семейного положения \n')

plt.draw()

**Вывод:**

Рассчитывать коэффициент корелляции мы не будем - выборка слишком мала, но в этом и нет необходимости. Графическое представление наглядно показывает нам разницу между количеством просрочек у людей, состоящих либо ранее состоявших в официальных отношениях, с людьми, которые в официальных отношениях не состоят. Сделать какой-то вывод о причинах такой зависимости сложно; возможно так происходит из-за более ответственного отношения к соблюдению формальностей у людей, оформлявших отношения. 

Таким образом, можно сделать общий вывод о том, что люди, не состоящие в официально оформленных отношениях, являются **менее надежными** заемщиками. 

Так же интересно, что разведенные и вдовцы - самые надежные плательщики. Причины этого лежат за пределами нашего исследования. 

<div class="alert alert-success">
<b>✔️ Комментарий ревьюера v1:</b>
<br>Абсолютно верно!Вывод очень полный и структурированный.
</div>

**Задание 21. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

### Зависимость между уровнем дохода и возвратом кредита в срок

Произведем схожие с предыдущими разделами операции по группировке и вычислению среднего процента задолженности в соответствующей группе. Построим диаграмму для более наглядного представления результатов. 

In [ ]:
# Ваш код будет здесь. Вы можете создавать новые ячейки.
# Формируем датафрeйм, сгруппировав данные по уровню дохода, сделав сделав его столбцом.
# Так же рассчитываем отношение невозвращенных кредитов к общему количеству кредитов в группе.
income_stat = data.groupby('total_income_category', as_index=False)['debt'].mean()

# Рассчитаем общее количество клиентов, сгруппировав их по уровню дохода
df = data.groupby('total_income_category', as_index = False)['debt'].count()

# Домножим полученные значения в столбце 'debt' на сто и округлим до двух знаков, получив процент невозврата 
# кредитов по группе с точностью до сотых процента.  
income_stat['debt'] = round(income_stat['debt'] * 100, ndigits=2)

#Переименуем названия категорий для лучшего восприятия данных
names = {'E': 'Доход до 30000 т.р.','D':  'Доход от 30001 до 50000 т.р.','C': 'Доход от 50001 до 200000 т.р.', 
'B': 'Доход от 200001 до 1000000 т.р.', 'A': 'Доход свыше 1 млн. руб.'}
income_stat['total_income_category'] = income_stat['total_income_category'].map(names)
df['total_income_category'] = df['total_income_category'].map(names)

print('Общее количество клиентов, сгруппированных по уровню дохода \n')
display(df)
print(separator)
print()

print('Процент невозвратов кредитов в зависимости от уровня дохода: \n')
display(income_stat)
print(separator)
print()

print('Построим диаграмму по категриям:')
ax = seaborn.barplot (data=income_stat, y='total_income_category', x='debt')

ax.set(xlabel='Процент просрочек',
       ylabel='',
       title='Процент просрочек в зависимости от дохода \n')

plt.draw()

**Промежуточный вывод:** 
В данных наблюдаются как ожидаемые результаты: наибольшее количество кредитов, невыплаченных в срок приходится на наименее обеспеченную группу клиентов, так и несколько неожиданные:
* вторая по количеству невыплат группа клиентов - группа клиентов со средним и высоким доходом, самая многочисленная по общему количеству клиентов. 
* наиболее "обязательная" группа клиентов - группа с доходом равным и чуть выше, чем средний доход по стране. 
* максимальный месячный доход клиента (более миллиона) не гарантирует выплаты кредита в срок. 

<div class="alert alert-success">
<b>✔️ Комментарий ревьюера v1:</b>
<br>Отличная визуализация!
</div>

Однако, если внимательно посмотреть на распределение по категориям, то можно уведить, что оно явно неравномерно. Попробуем перегруппировать клиентов в зависимости от дохода. 

Для получения границ диапазонов возьмем значение двадцать пятого, пятидесятого, семьдесят пятого персентилей. Назовем группы, определенные с помощью вычисленных значений: *Низкий доход, Доход ниже медианного, Доход выше медианного, Высокий доход*.

Отдельно возьмем группу с наивысшим доходом - 3 верхних процента из диапазона. 
Отдельно возьмем группу с наименьшим доходом - 3 нижних процента из диапазона. 

Сначала получим данные для определения граничных значений диапазонов. 


In [ ]:
# Рассчитаем и выведем значение для пятого персентиля
low_low_income = data.total_income.quantile(0.03)
print(f'Верхняя граница для отнесения заемщика к категории "Наименьший доход": {low_low_income:.02f}р.')
print(separator)
print()

# Рассчитаем и выведем значение для двадцать пятого персентиля
low_income = data.total_income.quantile(0.25)
print(f'Верхняя граница для отнесения заемщика к категории "Низкий доход": {low_income:.02f}р.')
print(separator)
print()

# Рассчитаем и выведем значение для пятидесятого персентиля
low_middle_income = data.total_income.quantile(0.50)
print(f'Верхняя граница для отнесения заемщика к категории "Доход ниже среднего": {low_middle_income:.02f}р.')
print(separator)
print()

# Рассчитаем и выведем значение для семьдесят пятого персентиля
high_middle_income = data.total_income.quantile(0.70)
print(f'Верхняя граница для отнесения заемщика к категории "Доход выше среднего": {low_middle_income:.02f}р.')
print(separator)
print()

# Рассчитаем и выведем значение для девяносто пятого персентиля
high_income = data.total_income.quantile(0.97)
print(f'Верхняя граница для отнесения заемщика к категории "Высокий доход": {high_income:.02f}р.')
print()

Создадим функцию для группировки и применим ее к данным

In [ ]:
# Объявление функции
def categorize_income_percentile(income):
    try:
        if income <= low_low_income:
            return 'F'
        elif low_low_income <= income <= low_income:
            return 'E'
        elif low_income <= income <= low_middle_income:
            return 'D'
        elif low_middle_income <= income <= high_middle_income:
            return 'C'
        elif high_middle_income < income < high_income:
            return 'B'
        elif income > high_income:
            return 'A'
    except:
        pass

# Группировка
data['total_income_category'] = data['total_income'].apply(categorize_income_percentile)

<div class="alert alert-success">
<b>✔️ Комментарий ревьюера v1:</b> Фенкция написана верно :)

In [ ]:
# Формируем датафрeйм, сгруппировав данные по уровню дохода, сделав сделав его столбцом.
# Так же рассчитываем отношение невозвращенных кредитов к общему количеству кредитов в группе.
income_stat = data.groupby('total_income_category', as_index=False)['debt'].mean()

# Рассчитаем общее количество клиентов, сгруппировав их по уровню дохода
df = data.groupby('total_income_category', as_index = False)['debt'].count()

# Домножим полученные значения в столбце 'debt' на сто и округлим до двух знаков, получив процент невозврата 
# кредитов по группе с точностью до сотых процента.  
income_stat['debt'] = round(income_stat['debt'] * 100, ndigits=2)

#Переименуем названия категорий для лучшего восприятия данных
names = {'F': f'Доход до {low_low_income:.02f} р., наименьший.',
         'E': f'Доход от {low_low_income:.02f} до {low_income:.02f}р., низкий',
         'D': f'Доход от {low_income:0.02f} до {low_middle_income:0.02f}р., ниже медианного',
         'C': f'Доход от {low_middle_income:0.02f} до {high_middle_income:0.02f}р., выше медианного', 
         'B': f'Доход от {high_middle_income:0.02f} до {high_income:0.02f}р., высокий', 
         'A': f'Доход свыше {high_income:0.02f}р., наивысший'}
income_stat['total_income_category'] = income_stat['total_income_category'].map(names)
df['total_income_category'] = df['total_income_category'].map(names)

print('Общее количество клиентов, сгруппированных по уровню дохода \n')
display(df)
print(separator)
print()

print('Процент невозвратов кредитов в зависимости от уровня дохода: \n')
display(income_stat)
print(separator)
print()

print('Построим диаграмму по категриям:')
ax = seaborn.barplot (data=income_stat, y='total_income_category', x='debt')

ax.set(xlabel='Процент просрочек',
       ylabel='',
       title='Процент просрочек в зависимости от дохода \n')

plt.draw()

<div class="alert alert-success">
<b>✔️ Комментарий ревьюера v1:</b> Здесь для вывода можно было создать отдельный датафрейм для красоты и простоты понимания. Плюс за визуализацию!

Распределение выглядит значительно более нормальным, однако, не наблюдаются выбросы, выявленные на предыдущей диаграмме. Сгруппируем данные в менее крупные группы, "нарезав" их на пятипроцентные интервалы.

In [ ]:
# Разобъем диапазон значений столбца total_income на двадцать равных интервалов. 
data['group_prc'] = pd.qcut(data['total_income'], 20)
# Сгруппируем, рассчитаем средний процент задолженности для каждой группы 
# и среднее и медиану для столбца total_income
income_stat = data.groupby('group_prc', as_index=False).agg({'debt':['mean'],'total_income':['mean', 'median', 'count']})

print('Полученные данные:')
print(income_stat)
print(separator)
print()

# Рассчитаем по выборке коэффициент корреляции Пирсона и p-value для медианы
r, p = pearsonr(income_stat['debt']['mean'], income_stat['total_income']['median'])
print(f'Коэффициент Пирсона для зависимости процента задолженности от медианы суммы общего дохода по группе = {r:.02f}, p-value = {p:.03}.')
print(separator)
print()

# Рассчитаем по выборке коэффициент корреляции Пирсона и p-value для среднего
r, p = pearsonr(income_stat['debt']['mean'], income_stat['total_income']['mean'])
print(f'Коэффициент Пирсона для зависимости процента задолженности от средней суммы общего дохода  по группе = {r:.02f}, p-value = {p:.03}.')
print(separator)
print()

<div class="alert alert-success">
<b>✔️ Комментарий ревьюера v1:</b> Можно сделать вывод по коэфициентам?

In [ ]:
# Соберем еще раз датафрейм
income_stat = data.groupby('group_prc', as_index=False)['debt'].mean()

# Домножим полученные значения в столбце 'debt' на сто и округлим до двух знаков, получив процент невозврата 
# кредитов по группе с точностью до сотых процента.  
income_stat['debt'] = round(income_stat['debt'] * 100, ndigits=2)

# Настроим вывод диаграммы
ax = seaborn.barplot(data=income_stat, x='group_prc', y='debt')

ax.set(xlabel='Процент просрочек',
       ylabel='',
       title='Процент просрочек в зависимости от дохода \n')
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

plt.draw()

**Вывод:**
В итоге исследования данных удалось установить наличие обратной зависимости вероятности возникновения задолженности от суммы дохода заемщика. Достоверность гипотезы подтверждается при помощи расчета коэффициента корреляции Пирсона, приведенного выше. 

Кроме того, наблюдаются локальный подъем вероятности возникновения задолженности до максимальных значений на в диапазоне от 65 до 95 т. р.

Далее вероятность удерживается на весьма высоких уровнях, вплоть до достижения границы дохода в 200 т. р., где она начинает снижаться, даже с учетом локальных выбросов в группе с доходом свыше 1 млн. руб. 

Таким образом нами доказано, что плательщики, входящие в группы с низкими и высокими доходами **являются более надежными**.

**Задание 22. Как разные цели кредита влияют на его возврат в срок?**

### Влияние целей на возврат кредита в срок

Произведем расчеты, сходные с приведенными выше, сгруппировав данные по целям получения кредита и вычислив средний процент невозвратов кредита в группах. Построим диаграмму для более наглядного представления расчетов. 

In [ ]:
# Ваш код будет здесь. Вы можете создавать новые ячейки.
# Формируем датафрeйм, сгруппировав данные по целям получения кредита, сделав сделав его столбцом.
# Так же рассчитываем отношение невозвращенных кредитов к общему количеству кредитов в группе.
goal_stat = data.groupby('purpose_category', as_index=False)['debt'].mean()

# Домножим полученные значения в столбце 'debt' на сто и округлим до двух знаков, получив процент невозврата 
# кредитов по группе с точностью до сотых процента.  
goal_stat['debt'] = round(goal_stat['debt'] * 100, ndigits=2)

print('Процент невозвратов кредитов в зависимости от целей получения кредита: \n')
print(goal_stat)
print(separator)
print()

print('Построим диаграмму по категриям:')
ax = seaborn.barplot (data=goal_stat, y='purpose_category', x='debt')

ax.set(xlabel='Процент просрочек',
       ylabel='',
       title='Процент просрочек в зависимости от цели получения кредита \n')

plt.draw()

**Вывод:** :
В данном случае все просто - две группы явных неплательщиков: приобретатели автомобиля и получающие образование. Можно предположить, что данные группы целей подвержены рискам по следующим причинам:
* автомобиль в РФ является предметом престижного потребления и, зачастую, клиенты приобретают автомобили из более высокой ценовой категории, нежели та, что они могут себе позволить. При этом, стоимость обслуживания такого автомобиля оказывается сопоставимой с суммой платежей по кредиту (страхование, ТО, потребление топлива и т. д.), что влечет за собой повышенную нагрузку на бюджет плательщика. 
* кредит на образование берется либо для получения первого образования, либо тогда, когда текущий уровень дохода или профессия прекращают удовлетворять клиента и он ищет пути для изменения жизненных обстоятельств. 

И в том и в другом случае финансовое положение заемщика представляется несколько неустойчивым.

Приобретатели недвижимости являются **наиболее обязательными** плательщиками. Интересно было бы дополнительно категоризировать данные для того, чтобы выяснить, в равной степени ли надежными плательщиками являются приобретатели недвижимости в личных и коммерческих целях. Однако, представленные данные не дают возможности однозначно категоризировать их по этому признаку - есть назначения такие, как "операции с жилой недвижимостью" и т. п., которые нельзя отнести с уверенностью к потребительскому либо коммерческому сектору.   

<div class="alert alert-success">
<b>✔️ Комментарий ревьюера v1:</b>
<br>Абсолютно верно!
</div>

**Задание 23. Приведите возможные причины появления пропусков в исходных данных.**

### Возможные причины появления пропусков в исходных данных

*Ответ:*

Для того чтобы понять, как правильно обработать пропуски, необходимо определить механизмы их формирования. Различают следующие 3 механизма формирования пропусков: MCAR, MAR, MNAR:

* MCAR (Missing Completely At Random) — механизм формирования пропусков, при котором вероятность пропуска для каждой записи набора одинакова. Например, если проводился социологический опрос, в котором каждому десятому респонденту один случайно выбранный вопрос не задавался, причем на все остальные заданные вопросы респонденты отвечали, то имеет место механизм MCAR. В таком случае игнорирование/исключение записей, содержащих пропущенные данные, не ведет к искажению результатов.
* MAR (Missing At Random) — на практике данные обычно пропущены не случайно, а ввиду некоторых закономерностей. Пропуски относят к MAR, если вероятность пропуска может быть определена на основе другой имеющейся в наборе данных информации (пол, возраст, занимаемая должность, образование…), не содержащей пропуски. В таком случае удаление или замена пропусков на значение «Пропуск», как и в случае MCAR, не приведет к существенному искажению результатов.
* MNAR (Missing Not At Random) — механизм формирования пропусков, при котором данные отсутствуют в зависимости от неизвестных факторов. MNAR предполагает, что вероятность пропуска могла бы быть описана на основе других атрибутов, но информация по этим атрибутам в наборе данных отсутствует. Как следствие, вероятность пропуска невозможно выразить на основе информации, содержащейся в наборе данных.

Рассмотрим различия между механизмами MAR и MNAR на примере.

Люди, занимающие руководящие должности и/или получившие образование в престижном вузе чаще, чем другие респонденты, не отвечают на вопрос о своих доходах. Поскольку занимаемая должность и образование сильно коррелируют с доходами, то в таком случае пропуски в поле доходы уже нельзя считать совершенно случайными, то есть говорить о случае MCAR не представляется возможным.

Если в наборе данных есть информация об образовании и должности респондентов, то зависимость между повышенной вероятностью пропуска в графе доходов и этой информацией может быть выражена математически, следовательно, выполняется гипотеза MAR. В случае MAR исключение пропусков вполне приемлемо.

Однако если информация о занимаемой должности и образовании у нас отсутствует, то тогда имеет место случай MNAR. При MNAR просто игнорировать или исключить пропуски уже нельзя, так как это приведет к значительному искажению распределения статистических свойств выборки.

**Источник:** [Обработка пропусков данных](https://loginom.ru/blog/missing), loginom.ru

<div class="alert alert-success">
<b>✔️ Комментарий ревьюера v1:</b>
<br>Супер, отлично, что используешь дополнительные источники!
</div>

**Задание 24. Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.**

### Медианное значение для пропусков

*Ответ:*

Медианное значение менее склонно реагировать на выбросы значений в данных. Прочие методы, такие как удаление или игнорирование пропусков в данных, заполнение предыдущим значением и проч. являются еще менее точными и надежными. 

Существует точка зрения, что наиболее надежным способом заполнения пропусков в данных является регрессионный метод - метод заполнения на основе эмпирически выведенной формулы, но он не всегда применим, особенно в условиях большого количества пропусков или возникновения неслучайных пропусков в данных. 

## Общий вывод.

В итоге проведенного исследования можно вывести некоторые закономерности. Наиболее надежными плательщиками являются заемщики, состоящие (или состоявшие) в официальных отношений, без детей с высоким доходом, приобретающие недвижимое имущество.

Дополнительно можно указать на следующие факты:
* заемщики, берущие кредит на свадьбу являются более надежными, чем берущие кредит на образование или приобретение автомобиля;
* кредит на приобретение автомобиля выданный холостому заемщику с зарплатой в диапазоне от 100 т. р. до 200 т. р. является одним из наиболее рискованных;
* среди заемщиков с небольшим размером дохода процент просрочки так же достаточно низкий, но при достижении нижней границы дохода (менее 30 т. р.) процент просрочки резко возрастает.

<div style="border:solid lightblue 3px; padding: 20px">
<b>Общий комментарий ревьюера:</b>

У тебя хорошая  работа. Каждый заголовок выделен, каждое рассуждение выделено, каждый пункт выделен. Хочется отметить так же упорядоченность и последовательность действий, нет скачков от одних пунктов к другим.
    
Из рекомендаций прошу выводить полученные значения после выполнения твоих действий. Так проще проверять будет тебе свою работу и проверяющему :)

В остальном все очень хорошо. Проект принимаю, успехов в дальнейшем!😊
</div>